In [1]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

import pandas as pd
import numpy as np
import utility_functions as uf
import utility_functions2 as uf2
import string
import itertools
import timeit
import scipy.spatial.distance as dist
import scipy.sparse as sp


In [2]:
sparse = sp.rand(30000, 30000, 0.03, format='csr', dtype=np.bool).astype(np.int_)
arr = np.array(sparse.todense())
arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [3]:
def sparse_jaccard(matrix):
    # sparse = sp.csr_matrix(matrix)
    cols_sum = sparse.getnnz(axis=1)
    ab = sparse * sparse.T

    # for rows
    aa = np.repeat(cols_sum, ab.getnnz(axis=1))
    # for columns
    bb = cols_sum[ab.indices]
    similarities = ab.copy()
    similarities.data = similarities.data / (aa + bb - ab.data)
    # sim = similarities.todense()
    # np.fill_diagonal(sim, 1)
    return similarities

In [4]:
%%time
sim = sparse_jaccard(sparse)

CPU times: user 7.36 s, sys: 10.5 s, total: 17.8 s
Wall time: 19 s


In [210]:
%%time
simuf = uf2.createSimilarityMatrix(pd.DataFrame(arr), 'jaccard', np.bool)

In [199]:
np.allclose(sim, simuf)

True

In [36]:
n, m = 2000, 1000
name_gen = map(''.join, itertools.islice(itertools.product('ABCDE',repeat=6), n))
attr_gen = map(''.join, itertools.islice(itertools.product('EDCBA',repeat=6), m))
names = list(name_gen)
attr = list(attr_gen)
data = np.random.randint(2, size=(n, m))
df = pd.DataFrame(data, index=names, columns=attr)

In [37]:
%%time
x = uf.createSimilarityMatrix(df, 'jaccard', np.bool)

CPU times: user 1.98 s, sys: 31.2 ms, total: 2.02 s
Wall time: 2.03 s


In [15]:
%%memit
n, m = 3000, 3000
name_gen = map(''.join, itertools.islice(itertools.product('ABCDE',repeat=6), n))
attr_gen = map(''.join, itertools.islice(itertools.product('EDCBA',repeat=6), m))
names = list(name_gen)
attr = list(attr_gen)

CPU times: user 3.94 s, sys: 31.2 ms, total: 3.97 s
Wall time: 3.97 s


In [3]:
%%memit
# data = np.triu(np.random.random((n, m)))
# data = (data + data.T) / 2
data = np.random.randint(2, size=(n, m))
data

peak memory: 149.56 MiB, increment: 56.05 MiB


In [4]:
#%%memit
df = pd.DataFrame(data, index=names, columns=attr)
df2 = df.astype('boolean')
#df

In [5]:
%%time
%%memit
x = uf.createSimilarityMatrix(df, 'jaccard', np.bool)

peak memory: 216.37 MiB, increment: 80.11 MiB
CPU times: user 3.97 s, sys: 125 ms, total: 4.09 s
Wall time: 4.2 s


In [6]:
%%time
%%memit
x2 = uf2.createSimilarityMatrix(df, 'jaccard')

peak memory: 277.59 MiB, increment: 61.21 MiB
CPU times: user 5.27 s, sys: 156 ms, total: 5.42 s
Wall time: 5.52 s


In [7]:
assert(np.allclose(x, x2))